# Imports

In [8]:
%run Imports.ipynb
name = 'Kred'

ImportError: Numba needs NumPy 2.2 or less. Got NumPy 2.3.

ImportError: Numba needs NumPy 2.2 or less. Got NumPy 2.3.

In [2]:
from pygam import LinearGAM, LogisticGAM, PoissonGAM, GammaGAM, s

In [3]:
import time
from copy import deepcopy
from itertools import product

from sklearn.model_selection import ParameterGrid, cross_val_predict
from sklearn.metrics import (
    confusion_matrix, accuracy_score, f1_score, roc_auc_score,
    mean_squared_error, mean_absolute_error, r2_score
)


In [4]:
# -------------------------------------------------------------------------
# 1. PARAM GRID  –  now also hosts `cv` and `verbose`
# -------------------------------------------------------------------------
param_grid = {
    "model_cls": [LinearGAM, LogisticGAM, PoissonGAM, GammaGAM],
    "n_splines": [406],          # constant but explicit
    "lam":       [1, 10, 50, 60, 75, 85, 90, 100],
    "max_iter":  [50, 100, 150, 200, 300, 400, 500],
    "cv":        [5],            # ← here
    "verbose":   [True],         # ← and here
}

In [30]:
param_grid['n_splines'][0]

406

# 1) Read in Files

In [5]:
key_featsubgroups = pd.read_json('../pickle/2_FS/' + name + '/key_featsubgroups.json', orient='records', lines=True)
df = pd.read_json('../pickle/2_FS/' + name + '/2_df_new_.json', orient='records', lines=True)

In [6]:
print(df[target].value_counts()/df.shape[0])
print('df_shape: ', df.shape)

arrears
1   0.646
0   0.354
Name: count, dtype: float64
df_shape:  (129457, 418)


In [ ]:
# # 1️⃣  read the archive
# #     ─────────────────
# # * `allow_pickle=True` is **required** if any value in the original
# #   dictionary was not a pure NumPy array (e.g. a list, a Python object,
# #   a scalar, a scikit-learn model wrapped in an object array, …).
# npz = np.load("../pickle/4_Model_Optuna/results_dict_updated.npz",
#               allow_pickle=True)

# # 2️⃣  turn it back into an ordinary Python dict
# #     ──────────────────────────────────────────
# # * Each entry in an .npz file is a NumPy array.
# # * If the array is 0-D (`shape == ()`) and has dtype=object,
# #   call `.item()` to extract the original Python object / scalar.
# results_dict = {
#     key: (arr.item()        # unwrap 0-D object array
#           if arr.shape == () and arr.dtype == object
#           else arr)         # leave normal arrays as-is
#     for key, arr in npz.items()
# }

In [ ]:
# with open('../pickle/4_Model_Optuna/results_dict_updated.pkl', 'rb') as f:
#     results_dict = pickle.load(f)

In [ ]:
with open('../../ANOR1/pickle/4_Model_Optuna/results_dict_updated_5.pkl', 'rb') as f:
    results_dict = pickle.load(f)

In [16]:
type(results_dict)

dict

In [22]:
results_dict['xgbc0']['classifier']

AttributeError: 'XGBModel' object has no attribute 'feature_weights'

AttributeError: 'XGBModel' object has no attribute 'feature_weights'

AttributeError: 'XGBModel' object has no attribute 'feature_weights'

# 2) Create Model prediction functions

## 2.1) Split dataset into train/testing while excluding demographic features

In [11]:
def split_data_4(df, key_featsubgroups=key_featsubgroups, target=target, test_size=0.2, random_state=42):
    """
    Splits the dataset into training and testing sets while excluding demographic features.

    Parameters:
    df (DataFrame): The dataset containing features and target variable.
    key_featsubgroups (DataFrame): A mapping of feature subgroups.
    target (str): The name of the target variable.
    test_size (float, optional): The proportion of the dataset to allocate for testing. Default is 0.2.
    random_state (int, optional): Random seed for reproducibility. Default is 42.

    Returns:
    tuple: X_train, X_test, y_train, y_test (training and testing datasets)
    """

    # Extract demographic features
    demo_feat = key_featsubgroups.loc[key_featsubgroups['subgroup'] == 'demo', 'list_features'].values[0]
    print("Demographic Features:", demo_feat)

    # Separate features (X) and target variable (y), excluding demographic features
    X = df.drop(columns=[target] + demo_feat)
    y = df[target]

    # Split the dataset into training (80%) and testing (20%) sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state
    )

    # Print dataset shapes
    print(f"Training Features Shape: {X_train.shape}")
    print(f"Training Labels Shape: {y_train.shape}")
    print(f"Testing Features Shape: {X_test.shape}")
    print(f"Testing Labels Shape: {y_test.shape}")

    return X_train, X_test, y_train, y_test

## 2.2) Model Training and prediction

In [33]:
from functools import reduce

In [34]:
# -------------------------------------------------------------------------
# 2. helper: detect regression vs. binary classification
# -------------------------------------------------------------------------
def _problem_type(y):
    u = np.unique(y)
    return "binary" if len(u) == 2 and u.dtype.kind in "ifu" else "regression"

# -------------------------------------------------------------------------
# 3. MAIN FUNCTION
# -------------------------------------------------------------------------
def gam(param_grid: dict, df, results_dict):
    """
    Exhaustive grid-search over GAM variants.
    `cv` and `verbose` are treated like any other hyper-parameter and
    must live in `param_grid`.
    Returns
    -------
    best_params : dict
    results_dict : dict   (updated in place)
    """
    X_train, X_test, y_train, y_test = split_data_4(df)   # your own splitter
    task = _problem_type(y_train)
    n_features = X_train.shape[1]

    # cache the 406-spline term list once
    term_cache = {}
    for n_spl in param_grid["n_splines"]:
        tl = [s(i, n_splines=n_spl) for i in range(n_features)]
        term_cache[n_spl] = reduce(lambda a, b: a + b, tl[1:], tl[0])
    best_score  = -np.inf if task == "binary" else np.inf
    best_params = None
    best_model  = None
    t0 = time.time()

    for params in ParameterGrid(param_grid):
        model_cls = params["model_cls"]
        cv        = params["cv"]
        verbose   = params["verbose"]

        # skip incompatible combos
        if task == "binary" and model_cls is not LogisticGAM:
            continue
        if task == "regression" and model_cls is LogisticGAM:
            continue

        model = model_cls(
            term_cache[params["n_splines"]],
            lam=params["lam"],
            max_iter=params["max_iter"]
        )

        if verbose:
            print(f"[GAM] {model_cls.__name__} | lam={params['lam']} "
                  f"| max_iter={params['max_iter']} | cv={cv}")

        model.fit(X_train, y_train)
        y_pred_cv   = cross_val_predict(model, X_train, y_train, cv=cv)
        y_pred_test = model.predict(X_test)

        if task == "binary":
            score = roc_auc_score(y_test, model.predict_mu(X_test))
        else:
            score = mean_squared_error(y_test, y_pred_test, squared=False)

        is_better = score > best_score if task == "binary" else score < best_score
        if is_better:
            best_score, best_params = score, deepcopy(params)
            best_model = deepcopy(model)
            if verbose:
                tag = "ROC-AUC" if task == "binary" else "RMSE"
                print(f" → new best ({tag}={score:.4f})")

    # ----------------- final evaluation & logging -----------------
    best_model.fit(X_train, y_train)
    y_train_pred = cross_val_predict(best_model, X_train, y_train,
                                     cv=best_params["cv"])
    y_test_pred  = best_model.predict(X_test)

    entry_key = f"gam_best_{best_model.__class__.__name__}"
    log = {
        "estimator": deepcopy(best_model),
        "best_params": best_params,
        "fit_time_s": round(time.time() - t0, 2),
    }

    if task == "binary":
        log.update({
            "cfm_train": confusion_matrix(y_train, y_train_pred),
            "cfm_test":  confusion_matrix(y_test,  y_test_pred),
            "train_accuracy": accuracy_score(y_train, y_train_pred),
            "test_accuracy":  accuracy_score(y_test,  y_test_pred),
            "test_roc_auc":   roc_auc_score(y_test, best_model.predict_mu(X_test)),
        })
    else:
        log.update({
            "rmse_test": mean_squared_error(y_test, y_test_pred, squared=False),
            "mae_test":  mean_absolute_error(y_test, y_test_pred),
            "r2_test":   r2_score(y_test, y_test_pred),
        })

    results_dict[entry_key] = log

    if best_params["verbose"]:
        metric = "ROC-AUC" if task == "binary" else "RMSE"
        print(f"\n✓ Stored best GAM under '{entry_key}' "
              f"({metric}={best_score:.4f}, "
              f"time={log['fit_time_s']:.1f}s)")

    return best_params, results_dict

# 6) GAM

## 6.1) Grid

In [ ]:
best_params, results_dict = gam(param_grid, df, results_dict)

Demographic Features: ['clientdata.demo.gender', 'clientdata.demo.age_year', 'clientdata.demo.age_month', 'clientdata.demo.children', 'clientdata.demo.children_singleparent', 'clientdata.demo.maritalstatus_expand_SINGLE', 'clientdata.demo.maritalstatus_expand_MARRIED', 'clientdata.demo.maritalstatus_expand_DIVORCED', 'clientdata.demo.maritalstatus_expand_WIDOWED', 'clientdata.demo.maritalstatus_expand_newvalue', 'clientdata.demo.maritalstatus_woe']
Training Features Shape: (103565, 406)
Training Labels Shape: (103565,)
Testing Features Shape: (25892, 406)
Testing Labels Shape: (25892,)
[GAM] LogisticGAM | lam=1 | max_iter=50 | cv=5


KeyboardInterrupt: 

## 6.2) Optuna 

In [ ]:
def gam_optuna(param_grid: dict, df, results_dict):
    """
    Optuna‐tunes a pygam model (LinearGAM or LogisticGAM),
    evaluates performance, and stores results in results_dict.

    Parameters:
    -----------
    param_grid : dict
        Search space with keys:
          - "model_cls": [LinearGAM, LogisticGAM]
          - "n_splines": list of ints
          - "lam": list of floats
          - "max_iter": list of ints
          - "cv": list of ints
          - "verbose": [True, False]
    df : DataFrame
        Input data.
    results_dict : dict
        Dictionary to log results.

    Returns:
    --------
    best_params : dict
        Best parameters found by Optuna.
    results_dict : dict
        Updated with evaluation metrics.
    """
    start_time = time.time()
    print("\nStarting Optuna hyperparameter search for GAM...")

    X_train, X_test, y_train, y_test = split_data_4(df)
    task = _problem_type(y_train)
    n_features = X_train.shape[1]

    # precompute term for given n_splines
    term_cache = {}

    def get_terms(n_splines):
        if n_splines not in term_cache:
            term_cache[n_splines] = sum(s(i, n_splines=n_splines) for i in range(n_features))
        return term_cache[n_splines]

    # choose study direction
    direction = "maximize" if task == "binary" else "minimize"
    study = optuna.create_study(direction=direction)

    def objective(trial):
        # sample from provided lists
        params = {k: trial.suggest_categorical(k, v) for k, v in param_grid.items()}
        Model = params["model_cls"]
        terms = get_terms(params["n_splines"])
        gam = Model(terms, lam=params["lam"], max_iter=params["max_iter"])
        if params["verbose"]:
            print(f"[GAM] Trial: {trial.number} | {Model.__name__} | lam={params['lam']} "
                  f"| max_iter={params['max_iter']} | cv={params['cv']}")

        gam.fit(X_train, y_train)
        # for classification use ROC-AUC on test
        if task == "binary":
            score = roc_auc_score(y_test, gam.predict_mu(X_test))
        else:
            y_pred = gam.predict(X_test)
            score = mean_squared_error(y_test, y_pred, squared=False)  # RMSE

        return score

    # estimate total trials
    total = np.prod([len(v) for v in param_grid.values()])
    print(f"No. of trials scheduled: {total}")
    study.optimize(objective, n_trials=int(total))

    best_params = study.best_params
    print("Best parameters:", best_params)

    # train best model
    Model = best_params["model_cls"]
    terms = get_terms(best_params["n_splines"])
    best_gam = Model(terms,
                     lam=best_params["lam"],
                     max_iter=best_params["max_iter"])
    best_gam.fit(X_train, y_train)

    # final predictions
    y_train_pred = cross_val_predict(best_gam, X_train, y_train, cv=best_params["cv"])
    y_test_pred  = best_gam.predict(X_test)

    # assemble log
    entry = f"gam_optuna_{Model.__name__}"
    log = {
        "estimator":   deepcopy(best_gam),
        "best_params": best_params,
        "fit_time_s":  round(time.time() - start_time, 2)
    }

    if task == "binary":
        log.update({
            "cfm_train":    confusion_matrix(y_train, y_train_pred),
            "cfm_test":     confusion_matrix(y_test,  y_test_pred),
            "train_accuracy": accuracy_score(y_train, y_train_pred),
            "test_accuracy":  accuracy_score(y_test,  y_test_pred),
            "test_roc_auc":   roc_auc_score(y_test, best_gam.predict_mu(X_test))
        })
    else:
        log.update({
            "rmse_test": mean_squared_error(y_test, y_test_pred, squared=False),
            "mae_test":  mean_absolute_error(y_test, y_test_pred),
            "r2_test":   r2_score(y_test, y_test_pred)
        })

    results_dict[entry] = log
    print(f"\n✓ Stored best GAM under '{entry}'")

    return best_params, results_dict

In [ ]:
best_params, results_dict = gam_optuna(param_grid, df, results_dict)